In [ ]:
import sys
sys.path.insert(0, '..')

import torch.nn as nn
from torch.utils.data import DataLoader
from utils.eye_dataset import *
from eye_classifier import *
import torchvision.transforms as transforms
import torchvision.models as models

### The Training Dataset

We use the utility class EyeImageDataset() to load the training dataset based on the metadata CSV file and
the target image folder

The training images will be loaded as needed due to memory constraints.

Since we're using RESNET-34 network as the convolutional layers, we need to resize our images to 224x224, so we apply
some transforms on our dataset.

We also need to proper normalize the dataset accordingly to the RESNET specification.

In [ ]:
base_dir = "../../data"
image_dir = f"{base_dir}/preprocessed_images"

image_dir_training = f"{base_dir}/ODIR-5K/training"
image_dir_testing = f"{base_dir}/ODIR-5K/testing"
csv_file = f'{base_dir}/ODIR-5K/data.csv'

print ('reading input dataset')
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_training, data_info_csv_file=csv_file, transform=apply_transforms)


### Building the model

To build the model, we use the utility superclass EyeClassifier. 

We create a subclass named ResnetEyeClassifier and then we feed it with the intended model on its __init__ constructor.

Here we're building a resnet34 as the first layer (in fact resnet18 is comprised of many layers), followed by three linear fully connected layers
for image classification

The last layer will not have a transfer function because we opted to use CrossEntropyLoss as the loss function. 

You can change the loss function by using set_loss_function() if you wish to

In [ ]:
class ResnetEyeClassifier(EyeClassifier):
    def __init__(self, num_classes: int) -> None:
        super(ResnetEyeClassifier, self).__init__(model=[

            (models.resnet34(pretrained=False), TransferFunction.NotApplicable),

            (nn.Linear(in_features=1000, out_features=256),
             TransferFunction.LeakyRelu),

            (nn.Linear(in_features=256, out_features=64),
             TransferFunction.LeakyRelu),

            (nn.Linear(in_features=64, out_features=16),
             TransferFunction.Relu),

            (nn.Linear(in_features=16, out_features=num_classes),
             TransferFunction.NotApplicable),
        ])


nn = ResnetEyeClassifier(num_classes=len(ds.classes))
print(nn)


### Training the model

Training the model is very easy. All we have to do is to call train_model() passing the EyeImageDataset object. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [ ]:
nn.train_model(ds, batch_size=16, num_epochs=30)

### Testing the model

Testing the model is similar to training it. All we have to do is to call test_model() passing a EyeImageDataset object pointing to the test dataset images. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [ ]:
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_testing, data_info_csv_file=csv_file, transform=apply_transforms)


In [ ]:
nn.test_model(ds)